## Libraries

In [1]:
import chess
import numpy as np
import random
import datetime
import pickle
import os

## Playground

In [2]:
newBoard=chess.Board()
newBoard.legal_moves
newBoard.is_check()
str(newBoard).split(" ")
newBoard.turn
str(newBoard).split("\n")

['r n b q k b n r',
 'p p p p p p p p',
 '. . . . . . . .',
 '. . . . . . . .',
 '. . . . . . . .',
 '. . . . . . . .',
 'P P P P P P P P',
 'R N B Q K B N R']

## Board Transformation Functions

In [3]:
#Piece Lookup
piece2int={'.':0,'P':chess.PAWN,'R':chess.ROOK,'N':chess.KNIGHT,'B':chess.BISHOP,'Q':chess.QUEEN,'K':chess.KING,
                             'p':chess.PAWN*-1,'r':chess.ROOK*-1,'n':chess.KNIGHT*-1,'b':chess.BISHOP*-1,'q':chess.QUEEN*-1,'k':chess.KING*-1}

domain=list(piece2int.values())
        
#Convert chess value to one_hot encoding
def create_one_hot(pieceId):
    one_hot=[]
    for i,j in enumerate(domain):
        if(j==pieceId):
            one_hot.append(1)
        else:
            one_hot.append(0)
    return np.array(one_hot)

#Construct tensor from matrix
def create_tensor(M):
    rows,cols=M.shape
    T=np.zeros((rows,cols,len(domain)), dtype=np.int)
    for i in range(rows):
        for j in range(cols):
            T[i,j]=create_one_hot(M[i,j])
    return T

#Convert list of list of string board to matrix
#board2Matrix=(lambda someBoard: np.matrix(list(map(lambda x:list(map(lambda y:y.replace(y,str(self.domain)), x.split(" "))), str(someBoard).split("\n"))), dtype=np.int))
def board2Matrix(someBoard):
    return np.matrix(list(map(lambda x:list(map(lambda y:y.replace(y,str(piece2int[y])), x.split(" "))), str(someBoard).split("\n"))), dtype=np.int)

#Convert matrix to tensor
#board2Tensor=(lambda someBoard: create_tensor(board2Matrix(someBoard), self.domain))
def board2Tensor(someBoard):
    return create_tensor(board2Matrix(someBoard))

## Intelligent Agent Class

Ability to save X,y, train, save, open, and predict from a DNN.

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


class Agent(object):
    
    model=None
    st_memory_X=[]
    st_memory_y=[]
    lt_memory=[]
    name=""
    def __init__(self,name):
        self.name=name
        self.build_model()

    def forget_st_X(self):
        self.st_memory_X=[]
    
    def forget_st_y(self):
        self.st_memory_y=[]
    
    def remember_st_X(self, point):
        self.st_memory_X.append(point)
        
    def remember_st_y(self, success):
        self.st_memory_y=[success for i in range(len(self.st_memory_X))]
    
    def remember_lt(self, fit=False):
        print("*** Committing to memory ***")
        y=np.array(self.st_memory_y, dtype=np.int)
        X=np.ndarray((len(self.st_memory_X), 8,8,13))
        for i in range(len(self.st_memory_X)):
            X[i]=self.st_memory_X[i]
            
        self.lt_memory.append((X,y))
        #if fit:
        self.fit_data(X,y)
        
        self.forget_st_X()
        self.forget_st_y()

    def build_model(self):
        
        model = Sequential()
        model.add(Dense(1028, activation='sigmoid', input_shape=(8,8,13)))
        model.add(Dense(1028, activation='tanh'))
        model.add(Dense(1028, activation='sigmoid'))
        model.add(Dense(1028, activation='tanh'))
        model.add(Dense(1028, activation='sigmoid'))
        model.add(Dense(1028, activation='tanh'))
        model.add(Dense(1028, activation='sigmoid'))
        model.add(Dense(1028, activation='tanh'))
        model.add(Dense(1028, activation='sigmoid'))
        model.add(Dense(1028, activation='tanh'))
        model.add(Dense(1028, activation='sigmoid'))
        model.add(Dense(1028, activation='tanh'))
        model.add(Dense(1028, activation='sigmoid'))
        model.add(Dense(1028, activation='tanh'))
        model.add(Dense(1028, activation='sigmoid'))
        model.add(Dense(1028, activation='tanh'))
        model.add(Dense(1028, activation='sigmoid'))
        model.add(Flatten())
        model.add(Dense(1028, activation='tanh'))
        model.add(Dense(1, activation='sigmoid'))

        model.compile(loss="categorical_crossentropy", 
                      optimizer=keras.optimizers.SGD(lr=0.01), 
                      metrics=['accuracy'])
        
        self.model=model
        return self.model
        
    def fit_data(self, Xtrain, ytrain):
        print("*** Learning from memory ***")
        print("AI " + self.name + "\tLearning.")
        assert(self.model!=None), "Model isnt built yet, run `build_model` first."
        self.model.fit(Xtrain, ytrain,
                  batch_size=10)
                  #epochs=epochs,
                  #verbose=1,
                  #validation_data=(Xtest, ytest),
                  #callbacks=[history]
                  #)

    
    def predict(self, point):
        return self.model.predict(np.expand_dims(point,axis=0))






/Users/laurensuarez/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Minimax and Alpha-Beta

with 5 heiristics

In [5]:
def material(board_state, weight):
    black_points = 0
    piece_values = {'p': 1, 'b': 3, 'n': 3, 'r': 5, 'q': 9, 'k': 0}
    for row in str(newBoard).split("\n"):
        for piece in row.split():
            if piece.islower():
                black_points -= piece_values[piece]
            elif piece.isupper():
                black_points += piece_values[piece.lower()]
    return black_points * weight

def piece_moves(game, weight):
    black_points = 0
    square_values = {"e4": 1, "e5": 1, "d4": 1, "d5": 1, "c6": 0.5, "d6": 0.5, "e6": 0.5, "f6": 0.5,
                    "c3": 0.5, "d3": 0.5, "e3": 0.5, "f3": 0.5, "c4": 0.5, "c5": 0.5, "f4": 0.5, "f5": 0.5}
    possible_moves = list(newBoard.legal_moves)
    for move in possible_moves:
        if newBoard.turn==True:
            if str(move)[2:4] in square_values:
                black_points += square_values[str(move)[2:4]]
        else:
            if str(move)[2:4] in square_values:
                black_points -= square_values[str(move)[2:4]]
    # piece_values = {'p': 1, 'b': 4, 'n': 4, 'r': 3, 'q': 3, 'k': 0}
    # for move in game.get_moves():
    #     current_piece = game.board.get_piece(game.xy2i(move[:2]))
    #     if current_piece.islower():
    #         black_points += piece_values[current_piece]
    return black_points

def pawn_structure(board_state, weight):
    black_points = 0

    # convert fen into matrix:
    board_state_arr = []
    for row in str(newBoard).split("\n"):
        row_arr = []
        for char in row:
            if char.isdigit():
                for i in range(int(char)):
                    row_arr.append(" ")
            else:
                row_arr.append(char)
        board_state_arr.append(row_arr)

    # determine pawn to search for based on whose turn it is
    for i, row in enumerate(board_state_arr):
        for j in range(len(row)):
            if newBoard.turn==True:
                if board_state_arr[i][j] == "P":
                    tl = i-1, j-1
                    tr = i-1, j+1
                    if tl[0] >= 0 and tl[0] <= 7 and tl[1] >= 0 and tl[1] <= 7:
                        if board_state_arr[tl[0]][tl[1]] == "P":
                            black_points += 1
                    if tr[0] >= 0 and tr[0] <= 7 and tr[1] >= 0 and tr[1] <= 7:
                        if board_state_arr[tr[0]][tr[1]] == "P":
                            black_points += 1
            else:
                if board_state_arr[i][j] == "p":
                    tl = i-1, j-1
                    tr = i-1, j+1
                    if tl[0] >= 0 and tl[0] <= 7 and tl[1] >= 0 and tl[1] <= 7:
                        if board_state_arr[tl[0]][tl[1]] == "p":
                            black_points -= 1
                    if tr[0] >= 0 and tr[0] <= 7 and tr[1] >= 0 and tr[1] <= 7:
                        if board_state_arr[tr[0]][tr[1]] == "p":
                            black_points -= 1
    return black_points * weight

def in_check(game, weight):
    black_points = 0
    current_status = int(newBoard.is_check())
    # Check or Checkmate situations
    if newBoard.turn == True:
        if current_status == 1:
            black_points += 1 * weight
        elif newBoard.is_checkmate():
            black_points += float("inf")
    else:
        if current_status == 1:
            black_points -= 1 * weight
        elif newBoard.is_checkmate():
            black_points += float("-inf")
    return black_points

def get_heuristic(board_state=None):
    if board_state == None:
        board_state = str(newBoard)
    clone = newBoard
    total_points = 0
    # total piece count
    total_points += in_check(clone, 1)
    total_points += pawn_structure(board_state, 1)
    total_points += material(board_state, 100)
    total_points += piece_moves(clone, 50)    
    return total_points


def ab_minimax(node, alpha, beta, current_depth=0):
    current_depth += 1
    if current_depth == 3:
        board_value = get_heuristic(newBoard)
        if current_depth % 2 == 0:
            # pick largest number, where root is black and even depth
            if (alpha < board_value):
                alpha = board_value
            return alpha
        else:
            # pick smallest number, where root is black and odd depth
            if (beta > board_value):
                beta = board_value
            return beta
    if current_depth % 2 == 0:
        # min player's turn
        for child_node in list(newBoard.legal_moves):
            if alpha < beta:
                newBoard.push(child_node)
                board_value = ab_minimax(child_node,alpha, beta, current_depth)
                newBoard.pop()
                if beta > board_value:
                    beta = board_value
        return beta
    else:
        # max player's turn
        for child_node in list(newBoard.legal_moves):
            if alpha < beta:
                newBoard.push(child_node)
                board_value = ab_minimax(child_node,alpha, beta, current_depth)
                newBoard.pop()
                if alpha < board_value:
                    alpha = board_value
        return alpha
    
def ab_make_move():
    possible_moves = list(newBoard.legal_moves)
    alpha = float("-inf")
    beta = float("inf")
    best_move = possible_moves[0]
    for move in possible_moves:
        board=newBoard.push(move)
        board_value = ab_minimax(move, alpha, beta, 1)
        newBoard.pop()
        if alpha < board_value:
            alpha = board_value
            best_move = move
            best_move.value = alpha
    # best_move at this point stores the move with the highest heuristic
    return best_move, board_value

## Pickle Load and Dump for MacOSFile Class

In [6]:
class MacOSFile(object):

    def __init__(self, f):
        self.f = f

    def __getattr__(self, item):
        return getattr(self.f, item)

    def read(self, n):
        # print("reading total_bytes=%s" % n, flush=True)
        if n >= (1 << 31):
            buffer = bytearray(n)
            idx = 0
            while idx < n:
                batch_size = min(n - idx, 1 << 31 - 1)
                # print("reading bytes [%s,%s)..." % (idx, idx + batch_size), end="", flush=True)
                buffer[idx:idx + batch_size] = self.f.read(batch_size)
                # print("done.", flush=True)
                idx += batch_size
            return buffer
        return self.f.read(n)

    def write(self, buffer):
        n = len(buffer)
        print("writing total_bytes=%s..." % n, flush=True)
        idx = 0
        while idx < n:
            batch_size = min(n - idx, 1 << 31 - 1)
            print("writing bytes [%s, %s)... " % (idx, idx + batch_size), end="", flush=True)
            self.f.write(buffer[idx:idx + batch_size])
            print("done.", flush=True)
            idx += batch_size

def pickle_dump(obj, file_path):
    with open(file_path, "wb") as f:
        return pickle.dump(obj, MacOSFile(f), protocol=pickle.HIGHEST_PROTOCOL)


def pickle_load(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(MacOSFile(f))

## Simulation

Training data shape needs to be:
board observation (tensor), move (string)
X in (8,8,16)
y in one_hot(length of entire state-space of moves)

In [14]:
os.chdir("/Users/laurensuarez/Documents/blake/turnouts/")

turnouts={'1-0':[],
          '0-1':[],
          '1/2-1/2':[]}

# Record 10,000 games each category
simCount=0
done=False
while True:
    
    memoryX=[]
    memoryY=[]
    newBoard = chess.Board()
    sequenceCount=0
    white_moves=[]
    black_moves=[]
    white_memory=[]
    black_memory=[]
    white_score=0
    black_score=0
    moves=[]
    memory=[]
    scores=[]
    while newBoard.is_game_over()==False:
        #move,points=ab_make_move()

        if sequenceCount%2==0:
            dataPoint=board2Tensor(newBoard)
            move=random.choice(list(newBoard.legal_moves))

            # Save data
            white_memory.append([dataPoint, str(move)])
            white_moves.append(move)
            white_score+=get_heuristic(move)
            newBoard.push(move)
        else:
            dataPoint=board2Tensor(newBoard)
            move=random.choice(list(newBoard.legal_moves))

            # Save data
            black_memory.append([dataPoint, str(move)])
            black_moves.append(move)
            black_score+=get_heuristic(move)
            newBoard.push(move)
        
        sequenceCount+=1

    winner=newBoard.result()
    
    if winner == '0-1':
        print("Black won.")
        turnouts[winner].append(black_memory)
        
        if len(turnouts[winner]) % 1000 == 0:
            print("Black just won 1000 games.")
        
    if len(turnouts['0-1']) == 10000:
        print("10,000 white games saved. ")
        file_path = "turnouts_10000(black)_"+str(datetime.date.today())+"_"+str(simCount)+".pkl"
        pickle_dump(turnouts, file_path)
        break


Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black just won 1000 games.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black

Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.

Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.

Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.

Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black won.
Black just

KeyboardInterrupt: 

## Write out games

In [18]:
#path=_4322_(black)_2018-11-18_0.pkl
file_path = "_"+str(len(turnouts['0-1']))+"_(black)_"+str(datetime.date.today())+"_"+str(simCount)+".pkl"
pickle_dump(turnouts, file_path)

## Read in games

In [19]:
d=pickle_load("_4322_(black)_2018-11-18_0.pkl")

In [29]:
x1=len(d['0-1'])#games
x2=len(d['0-1'][0])#first game
x3=len(d['0-1'][0][0])#tuple
x4=len(d['0-1'][0][0][0])#boardX
x5=len(d['0-1'][0][0][0][0])#boardY
x6=len(d['0-1'][0][0][0][0][0])#boardZ
print(x1)
print(x2)
print(x3)
print(x4)
print(x5)
print(x6)

4322
61
2
8
8
13


## Prepare the data

(N,8,8,16)

/Users/laurensuarez/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [30]:
X=list()
y=list()
i=0
for game in d['0-1']:
    for board,move in game:
        X.append(board)
        y.append(move)
X=np.array(X)

from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
y_one_hot = encoder.fit_transform(y)
y=encoder.inverse_transform(y_one_hot)
y_one_hot.shape

(365584, 1880)

In [31]:
X.shape

(365584, 8, 8, 13)

## Build the model

In [ ]:
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import Dropout
import keras
from keras.callbacks import History 
from sklearn.preprocessing import LabelBinarizer

def build_model(old_data, new_data):
    
    
    # Raw data conversion
    X=list()
    y=list()
    i=0
    if old_data != None:
        for game in old_data['0-1']:
            for board,move in game:
                X.append(board)
                y.append(move)
            
    for game in new_data['0-1']:
        for board,move in game:
            X.append(board)
            y.append(move)
    X=np.array(X)

    encoder = LabelBinarizer()
    y_one_hot = encoder.fit_transform(y)
    y=encoder.inverse_transform(y_one_hot)
    
    # Model construction
    model = Sequential()
    model.add(Dense(1028, activation='sigmoid', input_shape=(X.shape[1],X.shape[2],X.shape[3])))
    model.add(Conv2D(128, kernel_size=(3,3)))
    model.add(Dense(1028, activation='tanh'))
    model.add(Conv2D(256, kernel_size=(3,3)))
    model.add(Dense(1028, activation='sigmoid'))
    model.add(Conv2D(128, kernel_size=(3,3)))
    model.add(Dense(1028, activation='tanh'))
    model.add(Dense(1028, activation='sigmoid'))
    model.add(Dropout(0.2))
    model.add(Dense(1028, activation='tanh'))
    model.add(Dense(1028, activation='sigmoid'))
    model.add(Dense(1028, activation='tanh'))
    model.add(Dense(1028, activation='sigmoid'))
    model.add(Dense(1028, activation='tanh'))
    model.add(Dense(1028, activation='sigmoid'))
    model.add(Dense(1028, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(1028, activation='sigmoid'))
    model.add(Dense(1028, activation='tanh'))
    model.add(Dense(1028, activation='sigmoid'))
    model.add(Dense(1028, activation='tanh'))
    model.add(Dense(1028, activation='sigmoid'))
    model.add(Flatten())
    model.add(Dense(1028, activation='tanh'))
    model.add(Dense(y_one_hot.shape[1], activation='sigmoid'))
    model.compile(loss="categorical_crossentropy", 
                  optimizer=keras.optimizers.SGD(lr=0.01), 
                  metrics=['accuracy'])
    history = History()
    
    # Update weights
    model.fit(X, y_one_hot,batch_size=30,epochs=1,callbacks=[history])
    
    return model

def train_model(model, turnouts):
    
    # Raw data conversion
    X=list()
    y=list()
    i=0
    for game in old_data['0-1']:
        for board,move in game:
            X.append(board)
            y.append(move)
            
    for game in new_data['0-1']:
        for board,move in game:
            X.append(board)
            y.append(move)
    X=np.array(X)
    encoder = LabelBinarizer()
    y_one_hot = encoder.fit_transform(y)
    y=encoder.inverse_transform(y_one_hot)
    
    # Update weights
    model.fit(Xtrain, ytrain,
                  batch_size=10)
                  #epochs=epochs,
                  #verbose=1,
                  #validation_data=(Xtest, ytest),
                  #callbacks=[history]
                  #)
                
    return model

model=build_model(None, d)

Epoch 1/1
315300/365584 [========================>.....] - ETA: 28:29 - loss: 7.0028 - acc: 0.0071

In [ ]:
history

## Let our mini-ai play white

Reinforce will require a change in structure if perhaps a new move is introduced.

So we keep this as a proof of concept.

In [ ]:
os.chdir("/Users/laurensuarez/Documents/blake/turnouts/")

old_turnouts=d
turnouts={'1-0':[],
          '0-1':[],
          '1/2-1/2':[]}

numGames=1000
# Record 10,000 games each category
simCount=0
winCount=0
done=False
while True:
    
    newBoard = chess.Board()
    sequenceCount=0
    black_memory=[]
    reconstruct_model=False
    while newBoard.is_game_over()==False:
        # White move randomly
        if sequenceCount%2==0:
            move=random.choice(list(newBoard.legal_moves))
            newBoard.push(move)
        
        # Black respond intelligently
        else:
            # Create data point
            dataPoint=board2Tensor(newBoard)
            
            # Model prediction
            pred=model.predict(np.expand_dims(dataPoint,axis=0))[0]
            selectedIdx=np.argmax(pred)
            moveStr=encoder.inverse_transform(y_one_hot)[selectedIdx]
            
            # Convert prediction to string
            moveMake=None
            for move in list(map(lambda x:x, newBoard.legal_moves)):
                if str(move)==moveStr:
                    moveMake=move
            if moveMake!=None:
                black_memory.append([dataPoint, str(moveMake)])
                newBoard.push(moveMake)
                
            # Random Move if not in state-space
            else:
                print("Forced to move randomly. Rebuild model.")
                move=random.choice(list(newBoard.legal_moves))
                #black_memory.append([dataPoint, str(move)])
                newBoard.push(move)
            
            
        
        sequenceCount+=1
    
    
    winner=newBoard.result()
    print(winner)
    
    if winner == '0-1':
        winCount+=1
        print("Saving game.")
        turnouts[winner].append(black_memory)
        
    if winCount > 1000:
        model=train_model(model,turnouts)

